# Introduction

In the following Notebook we preprocess and clean each data file in order to extract the specific information we are seeking. Preprocessing actions are described in each cell on the comments. The aim is to have one combined dataframe containing all the data gathered from different sources in order to be used for prediction task.

## Note

All the dataframes have been saved as a pickle file after each step of preprocessing. This has been done for the simplicity. The pickle files are available under Data folder next to the other files of the project

# Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# Changing pandas display setting to be able to see all the columns

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_colwidth', 100)

# Load Datasets

For the sake of simplicity and making the code load faster, we already read the data and here we just load the saved pickle file

In [3]:
path = './Data/Pickle Files/'

In [4]:
disease_df = pd.read_pickle(path+'Disease_indicator_df.pickle')

In [5]:
pollution_annual_df = pd.read_pickle(path+'pollution_annual_df.pickle')

In [6]:
tax_df = pd.read_pickle(path+'Tax_df.pickle')

In [7]:
demog_df = pd.read_pickle(path+'population_df.pickle')

# Preprocessing

## Disease data

In [8]:
disease_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Year_Diff,Year
0,2014,2014,AR,Arkansas,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,1,2014
1,2018,2018,CO,Colorado,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN,1,2018
2,2018,2018,DC,District of Columbia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN,1,2018
3,2017,2017,GA,Georgia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN,1,2017
4,2010,2010,MI,Michigan,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,RACE,HIS,NaN,NaN,NaN,NaN,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185671,2020,2020,WY,Wyoming,BRFSS,Diabetes,Dilated eye examination among adults aged >= 1...,NaN,%,Age-adjusted Prevalence,...,DIA7_0,AGEADJPREV,RACE,WHT,NaN,NaN,NaN,NaN,1,2020
1185672,2020,2020,WY,Wyoming,BRFSS,Older Adults,Proportion of older adults aged >= 65 years wh...,NaN,%,Crude Prevalence,...,OLD3_1,CRDPREV,RACE,WHT,NaN,NaN,NaN,NaN,1,2020
1185673,2017,2017,IA,Iowa,BRFSS,Arthritis,Activity limitation due to arthritis among adu...,NaN,%,Age-adjusted Prevalence,...,ART2_1,AGEADJPREV,RACE,HIS,NaN,NaN,NaN,NaN,1,2017
1185674,2020,2020,WY,Wyoming,BRFSS,Diabetes,Diabetes prevalence among women aged 18-44 years,NaN,%,Crude Prevalence,...,DIA2_2,CRDPREV,RACE,HIS,NaN,NaN,NaN,NaN,1,2020


In [9]:
# Drop the unwanted columns

disease_df = disease_df.reset_index()
disease_df = disease_df.drop(['index','YearStart','YearEnd','DataSource','Response','DataValue','DataValueFootnoteSymbol','DatavalueFootnote','StratificationCategory2','Stratification2','StratificationCategory3','Stratification3','GeoLocation','ResponseID','LocationID','DataValueTypeID','StratificationCategoryID1','StratificationID1','StratificationCategoryID2','StratificationID2','StratificationCategoryID3','StratificationID3','Year_Diff'],axis=1)
disease_df.dropna(subset=['DataValueAlt'],inplace=True)

In [10]:
# Choose the overall stratification category among different categories
disease_df = disease_df[disease_df['StratificationCategory1']=='Overall']

In [11]:
# Choose diseases which could be related to the air pollution
disease_df = disease_df[disease_df['Topic'].isin(['Asthma','Cancer','Cardiovascular Disease','Chronic Obstructive Pulmonary Disease','Reproductive Health','Mental Health'])]

In [12]:
# Choose the column which is representing the number of incidents
disease_df = disease_df[disease_df['Question'].isin(['Cancer of the lung and bronchus, incidence','Cancer of the colon and rectum (colorectal), incidence','Invasive cancer of the oral cavity or pharynx, incidence','Invasive cancer of the female breast, incidence','Invasive cancer of the prostate, incidence','Invasive melanoma, incidence','Invasive cancer of the cervix, incidence']) ]

In [13]:
# Drop Nan Values
disease_df = disease_df[disease_df['DataValueUnit'].isna()==False]

In [14]:
# Pivot the table to desired way
disease_df = disease_df.pivot_table(index=['LocationAbbr','LocationDesc','Year'],columns='QuestionID',values='DataValueAlt').reset_index()
disease_df = disease_df.rename_axis(None, axis=1) 

In [15]:
# Renaming the columns
disease_df.columns = ['State','StateDesc','Year','CAN10_1','CAN11_1','CAN5_1','CAN6_1','CAN7_1','CAN8_1','CAN9_1']

In [16]:
# Save pickle file
# disease_df.to_pickle(".//Data//Pickle Files//Disease_Proc_df.pickle")  

## Pollution data

In [17]:
pollution_annual_df

,SITE_ID,SITE_NAME,STATE,COUNTY,LATITUDE,LONGITUDE,MAPID,YEAR,PRECIP,N_WET_NH4,...,MG_TOTAL,K_DRY,K_WET,K_TOTAL,CA_DRY,CA_WET,CA_TOTAL,CL_DRY,CL_WET,CL_TOTAL
0,ABT147,Abington,CT,Windham,41.840460,-72.010368,"Hampton, CT",2001.0,95.655,1.306,...,0.260,0.040,0.155,0.195,0.120,0.712,0.833,0.157,2.821,2.978
1,ABT147,Abington,CT,Windham,41.840460,-72.010368,"Hampton, CT",2002.0,114.430,1.749,...,0.289,0.045,0.269,0.314,0.114,0.724,0.838,0.157,2.901,3.058
2,ABT147,Abington,CT,Windham,41.840460,-72.010368,"Hampton, CT",2003.0,126.995,1.555,...,0.446,0.072,0.191,0.263,0.094,0.660,0.753,0.157,5.942,6.099
3,ABT147,Abington,CT,Windham,41.840460,-72.010368,"Hampton, CT",2004.0,126.351,1.540,...,0.300,0.043,0.143,0.186,0.091,0.571,0.662,0.224,3.686,3.910
4,ABT147,Abington,CT,Windham,41.840460,-72.010368,"Hampton, CT",2005.0,137.747,1.217,...,0.651,0.046,0.237,0.283,0.108,0.677,0.784,0.252,9.138,9.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,YOS404,Yosemite NP - Turtleback Dome,CA,Mariposa,37.713251,-119.706196,NaN,2016.0,111.357,1.176,...,0.148,0.084,0.135,0.219,0.171,0.438,0.608,0.130,1.004,1.135
2196,YOS404,Yosemite NP - Turtleback Dome,CA,Mariposa,37.713251,-119.706196,NaN,2017.0,169.564,1.305,...,0.159,0.136,0.122,0.259,0.244,0.336,0.579,0.102,1.491,1.594
2197,YOS404,Yosemite NP - Turtleback Dome,CA,Mariposa,37.713251,-119.706196,NaN,2018.0,90.299,1.012,...,0.159,0.220,0.113,0.333,0.488,0.373,0.861,0.095,0.809,0.905
2198,YOS404,Yosemite NP - Turtleback Dome,CA,Mariposa,37.713251,-119.706196,NaN,2019.0,125.660,1.856,...,0.194,0.080,0.227,0.307,0.199,0.547,0.746,0.073,1.799,1.872


In [18]:
# Drop unwanted columns
pollution_annual_df = pollution_annual_df.drop(['SITE_ID', 'SITE_NAME', 'COUNTY', 'MAPID', 'LATITUDE', 'LONGITUDE', 'LONGITUDE', 'N_TOTAL_NRED', 'N_TOTAL_NOXI', 'N_TOTAL', 'S_TOTAL', 'K_TOTAL', 'MG_TOTAL', 'NA_TOTAL', 'CA_TOTAL', 'CL_TOTAL'], axis=1)

In [19]:
# Group by year and state to have each row with yaer and state in a format which is desired
pollution_annual_df = pollution_annual_df.groupby(['YEAR', 'STATE']).mean().reset_index()

In [20]:
# Drop Nan values
pollution_annual_df = pollution_annual_df[pollution_annual_df['N_WET_NH4'].isna() == False]

In [21]:
pollution_annual_df

,YEAR,STATE,PRECIP,N_WET_NH4,N_WET_NO3,N_WET,N_DRY_HNO3,N_DRY_NO3,N_DRY_TNO3,N_DRY_NH4,...,NA_DRY,NA_WET,MG_DRY,MG_WET,K_DRY,K_WET,CA_DRY,CA_WET,CL_DRY,CL_WET
1,2001.0,AL,131.225000,2.506000,2.165000,4.67200,4.173000,0.286000,4.460000,0.398000,...,0.196000,1.27300,0.046000,0.22300,0.060000,0.551000,0.184000,0.866000,0.106000,2.245000
2,2001.0,AR,151.199000,1.507000,2.241000,3.74800,3.314000,0.077000,3.391000,0.233000,...,0.208000,1.44200,0.038000,0.24000,0.048000,0.526000,0.135000,1.754000,0.092000,2.448000
3,2001.0,AZ,34.533333,0.549667,0.827333,1.37700,1.498000,0.107333,1.605667,0.113667,...,0.126333,0.23300,0.035333,0.09500,0.038667,0.140667,0.269667,0.934667,0.044000,0.423667
4,2001.0,CA,58.153625,1.176000,0.892625,2.06875,3.764625,0.248125,4.012500,0.226250,...,0.288750,0.73975,0.058000,0.10500,0.070250,0.104875,0.261250,0.467375,0.088375,1.249750
5,2001.0,CO,55.322000,0.772000,1.045000,1.81650,1.412250,0.224000,1.636250,0.175000,...,0.095000,0.15750,0.038500,0.08650,0.048500,0.095000,0.275250,0.910750,0.048250,0.270750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,2020.0,VT,118.453500,1.962500,1.584000,3.54700,1.286000,0.105500,1.391500,0.141000,...,0.158500,0.65050,0.039000,0.23350,0.040500,0.347000,0.160500,1.258500,0.145500,1.145000
856,2020.0,WA,150.606750,0.770000,0.718250,1.48825,0.841250,0.044750,0.886000,0.074500,...,0.125750,2.65325,0.057500,0.31725,0.044750,0.256500,0.064500,0.469250,0.114250,4.828000
857,2020.0,WI,89.840000,4.528000,1.534000,6.06200,1.163000,0.140000,1.303000,0.143000,...,0.054000,0.30500,0.060000,0.35000,0.043000,0.269000,0.370000,3.310000,0.073000,0.512000
858,2020.0,WV,134.437000,1.895500,1.376500,3.27150,1.266000,0.070500,1.337000,0.145000,...,0.053000,0.39250,0.027500,0.15100,0.042000,0.234000,0.202500,1.186500,0.055500,0.655500


In [22]:
# change Year colum type to integer
pollution_annual_df['YEAR'] = pollution_annual_df.YEAR.astype(int)

In [23]:
# Rename the columns
pollution_annual_df = pollution_annual_df.rename(columns={'YEAR': 'Year', 'STATE': 'State'})

In [24]:
pollution_annual_df

,Year,State,PRECIP,N_WET_NH4,N_WET_NO3,N_WET,N_DRY_HNO3,N_DRY_NO3,N_DRY_TNO3,N_DRY_NH4,...,NA_DRY,NA_WET,MG_DRY,MG_WET,K_DRY,K_WET,CA_DRY,CA_WET,CL_DRY,CL_WET
1,2001,AL,131.225000,2.506000,2.165000,4.67200,4.173000,0.286000,4.460000,0.398000,...,0.196000,1.27300,0.046000,0.22300,0.060000,0.551000,0.184000,0.866000,0.106000,2.245000
2,2001,AR,151.199000,1.507000,2.241000,3.74800,3.314000,0.077000,3.391000,0.233000,...,0.208000,1.44200,0.038000,0.24000,0.048000,0.526000,0.135000,1.754000,0.092000,2.448000
3,2001,AZ,34.533333,0.549667,0.827333,1.37700,1.498000,0.107333,1.605667,0.113667,...,0.126333,0.23300,0.035333,0.09500,0.038667,0.140667,0.269667,0.934667,0.044000,0.423667
4,2001,CA,58.153625,1.176000,0.892625,2.06875,3.764625,0.248125,4.012500,0.226250,...,0.288750,0.73975,0.058000,0.10500,0.070250,0.104875,0.261250,0.467375,0.088375,1.249750
5,2001,CO,55.322000,0.772000,1.045000,1.81650,1.412250,0.224000,1.636250,0.175000,...,0.095000,0.15750,0.038500,0.08650,0.048500,0.095000,0.275250,0.910750,0.048250,0.270750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,2020,VT,118.453500,1.962500,1.584000,3.54700,1.286000,0.105500,1.391500,0.141000,...,0.158500,0.65050,0.039000,0.23350,0.040500,0.347000,0.160500,1.258500,0.145500,1.145000
856,2020,WA,150.606750,0.770000,0.718250,1.48825,0.841250,0.044750,0.886000,0.074500,...,0.125750,2.65325,0.057500,0.31725,0.044750,0.256500,0.064500,0.469250,0.114250,4.828000
857,2020,WI,89.840000,4.528000,1.534000,6.06200,1.163000,0.140000,1.303000,0.143000,...,0.054000,0.30500,0.060000,0.35000,0.043000,0.269000,0.370000,3.310000,0.073000,0.512000
858,2020,WV,134.437000,1.895500,1.376500,3.27150,1.266000,0.070500,1.337000,0.145000,...,0.053000,0.39250,0.027500,0.15100,0.042000,0.234000,0.202500,1.186500,0.055500,0.655500


In [25]:
# # Save pickle file
# pollution_annual_df.to_pickle(".//Data//Pickle Files//pollution_Proc_df.pickle")  

## Tax data

In [26]:
# Rename the columns
tax_df = tax_df.rename(columns={'Description': 'Tax_item_desc','item': 'Tax_item', 'States': 'State'})

In [27]:
# Replace the 'X' character in the tax column with Nan value in order to drop later and chage the type of the column to float
tax_df['Tax'] = tax_df['Tax'].replace('X', np.nan)
tax_df['Tax'] = tax_df['Tax'].astype(float)
tax_df['Year'] = tax_df['Year'].astype(int)

In [28]:
tax_df

,Tax_item_desc,Tax_item,Year,State,Tax
0,Property Taxes,T01,2000,WY,101396.0
1,Property Taxes,T01,2001,WY,110012.0
2,Property Taxes,T01,2002,WY,143975.0
3,Property Taxes,T01,2003,WY,146450.0
4,Property Taxes,T01,2004,WY,139809.0
...,...,...,...,...,...
27495,"Taxes, NEC",T99,2017,AL,0.0
27496,"Taxes, NEC",T99,2018,AL,0.0
27497,"Taxes, NEC",T99,2019,AL,0.0
27498,"Taxes, NEC",T99,2020,AL,0.0


In [29]:
# Group by year and state and sum over all different values of different tax types to calulate overal tax + drop unwanted columns

tax_df = tax_df.groupby(['Year', 'State']).agg('sum').reset_index()
tax_df = tax_df.drop(['Tax_item_desc', 'Tax_item'], axis=1)
tax_df

,Year,State,Tax
0,2000,AK,1423287.0
1,2000,AL,6438438.0
2,2000,AR,4870561.0
3,2000,AZ,8100737.0
4,2000,CA,83807959.0
...,...,...,...
1095,2021,VT,4102929.0
1096,2021,WA,32614091.0
1097,2021,WI,22300918.0
1098,2021,WV,6046197.0


In [30]:
# Save pickle file
# tax_df.to_pickle(".//Data//Pickle Files//tax_Proc_df.pickle")  

# Merging the dataframes

Below we merge all the preprocessed dataframes which we created above to have all the info about each year and state in one dataframe

In [31]:
merg_df = pd.concat([demog_df, tax_df.drop(columns=['Year', 'State'])], axis=1, join='outer')

In [32]:
merg1_df = pd.concat([disease_df, merg_df.drop(columns=['Year', 'State'])], axis=1)

In [33]:
merged_df = merg1_df.merge(pollution_annual_df, on = ['Year', 'State'], how = 'left')

In [34]:
# Save pickle file
# merged_df.to_pickle(".//Data//Pickle Files//merged_df.pickle")  

# Working with different Years and different states in each dataframe

Below we Filter the overlapping "States" among the different datasets and choose a subset of states which are present in all dataframes

In [35]:
s1 = set(disease_df['State'].unique())

In [36]:
s2 = set(pollution_annual_df['State'].unique())

In [37]:
diff = list(s1-s2)

In [38]:
disease_df = disease_df[~disease_df['State'].isin(diff)]
demog_df = demog_df[~demog_df['State'].isin(diff)]
tax_df = tax_df[~tax_df['State'].isin(diff)]

Filter the overlapping "Year" among the different datasets and choosing the data from 2001 to 2019 which are available in all datasets

In [39]:
tax_df = tax_df[tax_df['Year'].isin(list(range(2001, 2020)))]
pollution_annual_df = pollution_annual_df[pollution_annual_df['Year'].isin(list(range(2001, 2020)))]

since we could not get the demographic data from 2001 to 2009 we fill these missing values with the same values of year 2010

In [40]:
for y in range(2001, 2010):
    demog_2010 = demog_df[demog_df['Year'] == 2010].iloc[:,1:]
    demog_2010['Year'] = y
    demog_df = pd.concat([demog_df, demog_2010])

Final Merging:

In [41]:
merge_df = pd.concat([tax_df, demog_df.drop(columns=['Year', 'State'])], axis=1)

In [42]:
merge_df = pd.merge(merge_df, pollution_annual_df, on=['Year', 'State'])

In [ ]:
# Saving pickle files

# with open(path+'features.pickle', 'wb') as f:
#     pickle.dump(merge_df, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open(path+'disease_Proc_df.pickle', 'wb') as f:
#     pickle.dump(disease_df, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open(path+'pollution_Proc_df.pickle', 'wb') as f:
#     pickle.dump(pollution_annual_df, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open(path+'tax_Proc_df.pickle', 'wb') as f:
#     pickle.dump(tax_df, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open(path+'demog_Proc_df.pickle', 'wb') as f:
#     pickle.dump(demog_df, f, protocol=pickle.HIGHEST_PROTOCOL)

# Conclusion

Now we have one dataframe with all the needed info (e.g. demographic, polutants..) for each year and each state which will be used for training our model and conducting a prediction in the next notebook